In [18]:
import pandas as pd
import gc
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


df = pd.read_csv("../datasets/sell-in.txt.gz", sep="\t")
df = df.groupby(by=["periodo","product_id"]).agg({"tn":"sum"}).reset_index()
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df_pivot = df.pivot(index="periodo", columns="product_id", values="tn").reset_index()

magicos = [ "periodo",20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
   20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046,  20049,
   20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
   20193, 20320, 20532, 20612, 20637, 20807, 20838
 ]
X_train = df_pivot[magicos].query("periodo >= '2018-01-01' & periodo <= '2018-12-31' ")
X_train = X_train.T.iloc[1:]
X_train.columns = [f"t-{11-k}" for k in range(12)]


X_kgl = df_pivot.query("periodo >= '2018-03-01' & periodo <= '2019-02-28'")
X_kgl = X_kgl.T.iloc[1:]
X_kgl.columns = [f"t-{11-k}" for k in range(12)]
promedio = X_kgl.mean(axis=1)


y = df_pivot[magicos].query("periodo == '2019-02-01'").T.iloc[1:]
y.columns = ["target"]
y["target"] = pd.to_numeric(y["target"], errors='coerce')
y_log = np.log1p(y["target"])



C:\Users\carre\AppData\Local\Temp\ipykernel_10700\1055899961.py:30: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  y = df_pivot[magicos].query("periodo == '2019-02-01'").T.iloc[1:]


In [20]:
X_train

,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0
product_id,,,,,,,,,,,,
20002,984.80167,712.00087,966.86044,999.20934,1103.39191,1033.82845,977.40239,1161.8843,954.23575,1378.49032,1766.81068,1009.45458
20003,907.56304,788.30749,778.55594,765.47838,784.35885,660.73323,656.227,955.97079,912.34156,1313.34211,1206.91773,769.82869
20006,470.33785,588.56272,862.19361,748.44391,865.28861,468.1526,515.20419,615.70617,478.04388,513.15472,566.66809,407.75925
20010,298.25586,477.48363,653.77607,171.74107,227.24082,223.87746,331.96807,582.83104,480.60235,612.50721,414.97753,285.02947
20011,340.75314,601.26066,526.99374,562.70214,484.04538,437.7555,300.26178,191.0727,189.5985,177.75576,289.13976,321.09714
20018,342.16945,337.54792,510.33171,543.06908,388.43687,420.33781,325.03223,375.61778,282.43485,161.58557,371.52958,326.01506
20019,243.71984,629.89543,321.26878,290.39581,356.42982,262.33076,351.0561,387.73155,417.95455,552.71975,532.98143,446.69747
20021,253.48383,234.06376,405.9533,94.17409,153.29935,447.59775,332.17776,259.45932,391.53891,404.8525,313.67485,228.25426
20026,250.81615,197.61244,203.14786,317.66739,378.08331,317.87268,330.87172,352.77241,397.8142,328.17556,307.11434,223.97681


In [21]:
y

,target
product_id,
20002,1043.01349
20003,758.32657
20006,479.99914
20010,337.76009
20011,431.62938
20018,265.84135
20019,323.66178
20021,219.88515
20026,263.67957


In [22]:
promedio

product_id
20001    1532.718593
20002     1138.44743
20003     877.237503
20004     656.581815
20005     574.105717
            ...     
21295            NaN
21296            NaN
21297            NaN
21298            NaN
21299            NaN
Length: 1233, dtype: object

In [23]:
prod_menos12 = X_kgl.index[X_kgl.isna().sum(axis=1)> 0]
X_kgl = X_kgl[~X_kgl.index.isin(prod_menos12)]
promedio_menos12 = promedio[prod_menos12]

HIPERPARAMETROS

In [24]:
#predict 
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")


In [25]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression()
reg_model.fit(X_train, y_log)

LinearRegression()

In [26]:
reg_model.intercept_

np.float64(2.7813438197869735)

In [27]:
reg_model.coef_

array([-0.00321642,  0.00484371, -0.00023674, -0.00025366,  0.0008579 ,
        0.00680834, -0.00201161,  0.00096855,  0.00722186, -0.00202726,
       -0.00098068, -0.00520164])

In [28]:
reg_model.coef_

array([-0.00321642,  0.00484371, -0.00023674, -0.00025366,  0.0008579 ,
        0.00680834, -0.00201161,  0.00096855,  0.00722186, -0.00202726,
       -0.00098068, -0.00520164])

In [29]:
X_kgl

,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0
product_id,,,,,,,,,,,,
20001,1856.83534,1251.28462,1293.89788,1150.79169,1470.41009,1800.96168,1438.67455,2295.19832,1813.01511,1486.68669,1275.77351,1259.09363
20002,966.86044,999.20934,1103.39191,1033.82845,977.40239,1161.8843,954.23575,1378.49032,1766.81068,1009.45458,1266.78751,1043.01349
20003,778.55594,765.47838,784.35885,660.73323,656.227,955.97079,912.34156,1313.34211,1206.91773,769.82869,964.76919,758.32657
20004,488.92473,611.51237,641.37063,447.84475,653.4231,936.42001,948.86342,809.67086,802.34669,585.56477,511.33713,441.70332
20005,559.98671,496.41774,637.11135,547.62513,502.34077,874.88924,761.7752,893.74086,469.26344,372.63428,363.58438,409.8995
...,...,...,...,...,...,...,...,...,...,...,...,...
21222,0.06919,0.09173,0.0983,0.08301,0.09829,0.09394,0.06261,0.08006,0.0415,0.02839,0.06478,0.03786
21224,0.11667,0.10316,0.072,0.18093,0.08918,0.06555,0.06121,0.06258,0.05231,0.07982,0.09469,0.08571
21255,0.01641,0.03278,0.01705,0.04542,0.03412,0.03255,0.0758,0.05854,0.02206,0.00613,0.00852,0.01638


In [30]:
len(X_kgl.index)

769

In [31]:
pred = pd.DataFrame({"product_id": X_kgl.index, "tn": reg_model.predict(X_kgl).flatten()})
nuevas_filas = []
for prod in productos_ok["product_id"]:
    if prod not in pred["product_id"].values:
        nuevas_filas.append({"product_id": prod, "tn": promedio[prod]})

pred = pd.concat([pred, pd.DataFrame(nuevas_filas)], ignore_index=True)
pred = pred[pred["product_id"].isin(productos_ok["product_id"])]
pred

,product_id,tn
0,20001,17.402566
1,20002,16.198083
2,20003,12.404838
3,20004,11.525115
4,20005,8.960515
...,...,...
974,21263,0.216322
975,21265,NaN
976,21266,NaN
977,21267,NaN


In [32]:
pred.to_csv("../results/reg_lineal_log.csv", index=False, sep=",")